# GenderME

**Notes on the .txt:** 
1. Schäuble announces speakers. (Relevant for finding the beginning of new speeches?)
2. Interjections from other politicians marked with `(...)` -- for interjections, party affiliation is marked with `[...]`.  
3. The begin of a speech is marked with ':' - however, these also appear in speeches. For normal speakers, party affiliation is indicated with `(...)`.  

**Needed packages:**

In [1]:
options(warn=-1)

install.packages("readr")
install.packages("tidyverse")

package 'readr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Arilila\AppData\Local\Temp\RtmpUlYAHB\downloaded_packages
package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\Arilila\AppData\Local\Temp\RtmpUlYAHB\downloaded_packages


In [16]:
library("readr")
library("tidyverse")

Load test file `plenartest.txt`.

In [37]:
plenar_test <- read_file("res/plenartest.txt")

Working with the .txt file:

Extract interjections `(...)`. 

After this step, we have our variable `plenar_test` without interjections and a second variable named `interjections` with only the interjections.

In [90]:
parenthesis_pattern <- "(?=\\().*?(?<=\\))(apple|orange|juice)(?:[^-a-z0-9A-Z_]|$)"

interjections <- regmatches(
    plenar_test, 
    gregexpr(parenthesis_pattern, plenar_test, perl=T),
    invert = FALSE
)[[1]]

plenar_test <- regmatches(
    plenar_test, 
    gregexpr(parenthesis_pattern, plenar_test, perl=T), 
    invert = TRUE
)[[1]]

In [39]:
plenar_test <- paste(plenar_test, collapse = '')

This command filters the interjections. The output are interjections with dialogue elements. (This might be useful for later analysis.)

In [40]:
# for(interjection in interjections){
#     if(grepl(':', interjection, fixed = TRUE)){
#         print(interjection)
#     } 
# }

Find all instances of `Präsident Dr. Wolfgang Schäuble:` in the .txt file. This helps us:
1. Find the beginning of the relevant text, allowing us to purge the meaningless beginning.
2. Find the beginning of the individual speeches, since Schäuble as speaker of the plenary session has to introduce all speakers.

In [79]:
plenar_test_vec <- strsplit(plenar_test, "Präsident Dr. Wolfgang Schäuble:")[[1]];
plenar_test_vec <- plenar_test_vec[2:length(plenar_test_vec)]